# Lab 02 Extended Tasks

## Part 1: Save RSSFeed Content into a DataFrame

In [1]:
import requests
import feedparser
from bs4 import BeautifulSoup
import pandas as pd

listOfArticles = [] #list will be used to construct the dataframe

def getArticleDetailsByUrl(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,"html.parser")
    
    headline = soup.title.string #extract the headline of the article

    content = ''
    body = soup.find("div", {"class": "article_bodycopy"}) #the div tag with class=article_bodycopy contains all the text
    p_tags = soup.findAll("p") #each p tag contains a sentence of the article
    for p in p_tags:
        content += p.get_text() #append the text of each p tag to the string to record all of the article content

    return [headline, content]

def scrapeRSSFeed(rss_feed): #rss_feed is the url of a website's RSS feed/XML file
    RSSdict = feedparser.parse(rss_feed)  #create a feedparser dictionary holding all of the info
    
    for item in RSSdict['entries']: #for each entry in the dict
        article_url = item['link'] #get the link/URL of the article
        [headline, content] = getArticleDetailsByUrl(article_url) #record the details of each article using the function above
        listOfArticles.append([headline, content]) #append to list to use in dataframe

In [2]:
url='http://www.irishtimes.com/cmlink/news-1.1319192' #RSS of IrishTimes
scrapeRSSFeed(url) #use above function to gather the info

df = pd.DataFrame(listOfArticles) #make a dataframe from the list
df.columns = ["headline", "content"]

In [3]:
df #display the dataframe

,headline,content
0,EU Contest for Young Scientists features starc...,EU Contest for Young Scientists winners: ...
1,Junior entrepreneurs encouraged to start real ...,The Junior Entrepreneur Programme (JEP) ru...
2,New personal injury measures needed to reduce ...,Nicholas Kearns: report points out that “e...
3,Councils should buy vacant houses compulsorily...,The Indecon’s recommendations will inform ...
4,Unemployment main driver for Spaniards moving ...,"Alvaro Cabello, who now teaches Padel, the..."
5,"Surgical mesh failure rates unacceptably high,...",Many women in Ireland are experiencing med...
6,Man (83) ‘hadn’t much choice’ but to try stop ...,A recording of CCTV footage appears to sho...
7,Minister cautions that report on future of pol...,"Kathleen O’Toole, chairwoman of the Commis..."
8,Cervical cancer vaccine uptake rises to 65%,Minister for Health Simon Harris at the la...
9,"Lock up your tractors, gardaí warn, as one in ...",Crowds at the opening day of the National...


## Part 2: Extract 10 Tweets about #analytics and store in a dataframe 

In [4]:
#initialise json and twitter settings
import json
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream

ACCESS_TOKEN = '2839893905-pBXUzdrHCNXyjfPuBpSwxNbH1zyEpRaa2sXK0Jd'
ACCESS_SECRET = 'eNtB7YTAfsMhPIQtKji8aQT7zQFpFfDPR2lQ89WKfgI1U'
CONSUMER_KEY = 'ZqPrfLpc0znZlz3kW2a22VmUa'
CONSUMER_SECRET = 'BHD19T0DmUV2XVvEhUAgvpXMx0nGfxevAtr53NbCd9jQjPyTqn'

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter = Twitter(auth=oauth)
            
# Search for latest 10 tweets about "#analytics"
iterator = twitter.search.tweets(q='#analytics', result_type='recent', lang='en', count=10)

#save this to a JSON
file = open("twitter_search_10tweets.json", "w") 
for tweet in iterator['statuses']:
    file.write(json.dumps(tweet)+"\n")
file.close()

{"created_at": "Tue Sep 18 15:53:20 +0000 2018", "id": 1042079130467160069, "id_str": "1042079130467160069", "text": "RT @antgrasso: The purpose of a data pipeline is to move data from a point of origin to a specific destination. At a high level, a data pip\u2026", "truncated": false, "entities": {"hashtags": [], "symbols": [], "user_mentions": [{"screen_name": "antgrasso", "name": "Antonio Grasso", "id": 4338452835, "id_str": "4338452835", "indices": [3, 13]}], "urls": []}, "metadata": {"iso_language_code": "en", "result_type": "recent"}, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 779513335393968128, "id_str": "779513335393968128", "name": "Ene", "screen_name": "ene3112", "location": "", "description": "Noticias", "url": null, "entities": {"description":

In [5]:
#opemn JSON file just saved
with open('twitter_search_10tweets.json', 'r') as f:
    tweets_file = f.readlines()

listOfInfo = [] #list used to construct dataframe

for line in tweets_file:
    try:
        tweet = json.loads(line.strip())
        if 'text' in tweet: # only messages contains 'text' field is a tweet
            userID = tweet['user']['id'] #user who tweeted
            text = tweet['text'] #text of tweet
            rtCount = tweet['retweet_count'] #number of retweets
            try:
                userRT = tweet['retweeted_status']['in_reply_to_user_id'] #who was this retweeted from?
            except:
                userRT = None #if not present, was not retweeted
    
            listOfInfo.append([userID, text,rtCount,userRT]) #append info to list
    except:
        print("JSON error!!!")
        continue

In [6]:
df2 = pd.DataFrame(listOfInfo) #construct dataframe
df2.columns = ["UserID", "Text", "RTcount", "userRT"]
df2 #display dataframe

,UserID,Text,RTcount,userRT
0,779513335393968128,RT @antgrasso: The purpose of a data pipeline ...,50,None
1,875441049061031936,RT @ahmedjr_16: 17 Best #IT #Training &amp; #C...,2,None
2,875441049061031936,RT @ahmedjr_16: 11 #BigData Courses To Become ...,8,None
3,1009528562511011840,Big Data - The 8 V's Via @alison_iot #DigitalM...,0,None
4,47985748,One last tweet for today on the #railanalytics...,0,None
5,1550350116,Looking to establish an #analytics function in...,0,None
6,875441049061031936,RT @ahmedjr_16: 48 Best #Development Courses \...,9,None
7,930871517860319232,RT @KirkDBorne: #MachineLearning with Signal P...,32,None
8,875441049061031936,RT @ahmedjr_16: 8 Best SQL Courses Online To A...,12,None
9,875441049061031936,RT @ahmedjr_16: 29 Best #DataAnalytics Courses...,1,None
